## Daysim Person and Household Models

*Summaries:*
    - Total People & Workers
    - Household Size
    - County Population
    - Age Distribution
    - Worker Types
    - Distance to Work
    - Transit Pass Ownership
    - Vehicle Ownership
---

In [120]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')

In [121]:
import os
import numpy as np
import pandas as pd
import h5py
import pylab as P
from IPython.display import display, display_pretty, Javascript, HTML
from pandas_highcharts.core import serialize
from pandas_highcharts.display import display_charts
import matplotlib.pyplot as plt

# Show charts in notebook
# %pylab inline

In [122]:
# Set main model directory to parent directory

# 2006 survey
model_dir = r'R:\SoundCast\releases\TransportationFutures2010\scripts\summarize'

# 2014 survey
base_dir = r'R:\SoundCast\releases\TransportationFutures2010\scripts\summarize' 

In [123]:
# Read Model Scenario Results
scen = h5py.File(model_dir + r'/survey.h5','r+')
scen_name = '2006 Survey'

In [124]:
# Read Base Data
base_file = r'/survey14.h5'

base = h5py.File(base_dir + base_file ,'r+')
base_name = '2014 Survey'

In [351]:
# Get person data into a single dataframe for base and scen
data_table = 'Person'

person_scen = pd.DataFrame(data={'Household ID': [i[0] for i in scen[data_table]['hhno'][:]],
                               'Person Number': [i[0] for i in scen[data_table]['pno'][:]],
                               'Transit Pass': [i[0] for i in scen[data_table]['ptpass'][:]],
                                'Auto Time to Work': [i[0] for i in scen[data_table]['pwautime'][:]],
                                'Auto Distance to Work': [i[0] for i in scen[data_table]['pwaudist'][:]],
                                'Worker Type': [i[0] for i in scen[data_table]['pwtyp'][:]],
                                'Student Type': [i[0] for i in scen[data_table]['pstyp'][:]],
                                'Usual Commute Mode': [i[0] for i in scen[data_table]['pwtaz'][:]],
                                'Workplace TAZ': [i[0] for i in scen[data_table]['pwtaz'][:]],
                                'Age': [i[0] for i in scen[data_table]['pagey'][:]],
                                'Expansion Factor': [i[0] for i in scen[data_table]['psexpfac'][:]]})

person_base = pd.DataFrame(data={'Household ID': [i[0] for i in base[data_table]['hhno'][:]],
                                 'Person Number': [i[0] for i in base[data_table]['pno'][:]],
                                 'Transit Pass': [i[0] for i in base[data_table]['ptpass'][:]],
                                 'Auto Time to Work': [i[0] for i in base[data_table]['ptpass'][:]],
                                 'Auto Distance to Work': [i[0] for i in base[data_table]['pwautime'][:]],
                                 'Worker Type': [i[0] for i in base[data_table]['pwtyp'][:]],
                                 'Student Type': [i[0] for i in base[data_table]['pstyp'][:]],
                                 'Usual Commute Mode': [i[0] for i in base[data_table]['puwmode'][:]],
                                 'Workplace TAZ': [i[0] for i in base[data_table]['pwtaz'][:]],
                                 'Age': [i[0] for i in base[data_table]['pagey'][:]],
                                 'Expansion Factor': [i[0] for i in base[data_table]['psexpfac'][:]]})

In [352]:
# Create unique ID for person by concatenating household ID and person number 
person_scen['personID'] = (person_scen['Household ID'].astype('str')+person_scen['Person Number'].astype('str')).astype('int')
person_base['personID'] = (person_base['Household ID'].astype('str')+person_base['Person Number'].astype('str')).astype('int')

In [353]:
# Get household data into a single dataframe for base and scen
data_table = 'Household'

hh_scen = pd.DataFrame(data={'Household ID': [i[0] for i in scen[data_table]['hhno'][:]],
                                'Household Size': [i[0] for i in scen[data_table]['hhsize'][:]],
                                'Household Vehicles': [i[0] for i in scen[data_table]['hhvehs'][:]],
                                 'Household Workers': [i[0] for i in scen[data_table]['hhwkrs'][:]],
                                 'Household Income': [i[0] for i in scen[data_table]['hhincome'][:]],
                                 'Household TAZ': [i[0] for i in scen[data_table]['hhtaz'][:]],
                                 'Expansion Factor': [i[0] for i in scen[data_table]['hhexpfac'][:]]})

hh_base = pd.DataFrame(data={'Household ID': [i[0] for i in base[data_table]['hhno'][:]],
                                 'Household Size': [i[0] for i in base[data_table]['hhsize'][:]],
                                 'Household Vehicles': [i[0] for i in base[data_table]['hhvehs'][:]],
                                 'Household Workers': [i[0] for i in base[data_table]['hhwkrs'][:]],
                                 'Household Income': [i[0] for i in base[data_table]['hhincome'][:]],
                                 'Household TAZ': [i[0] for i in base[data_table]['hhtaz'][:]],
                                 'Expansion Factor': [i[0] for i in base[data_table]['hhexpfac'][:]]})

In [354]:
# Add labels for worker type
labels = {
  0: "Not a worker",  
  1: "Full-time worker",
  2: "Part-time worker",
}

person_base['Worker Type'] = ([labels[x] for x in person_base['Worker Type']])
person_scen['Worker Type'] = ([labels[x] for x in person_scen['Worker Type']])

In [355]:
# # Add labels for student type
# labels = {
#   0: "Not a student",  
#   1: "Full-time student",
#   2: "Part-time student",
# }

# person_base['Student Type'] = ([labels[x] for x in person_base['Student Type']])
# person_scen['Student Type'] = ([labels[x] for x in person_scen['Student Type']])

In [356]:
# Join household records to person records
hh_per_scen = pd.merge(left=person_scen, right=hh_scen,on='Household ID',suffixes=('_p','_h'))
hh_per_base = pd.merge(left=person_base, right=hh_base,on='Household ID',suffixes=('_p','_h'))

In [357]:
# Join household geography
taz_geog = pd.read_csv(r'utils/taz_lookup.csv')
taz_geog.reindex
hh_per_scen_home_geog = pd.merge(hh_per_scen, taz_geog, left_on='Household TAZ', right_on='TAZ')
hh_per_base_home_geog = pd.merge(hh_per_base, taz_geog, left_on='Household TAZ', right_on='TAZ')

In [358]:
# Join workplace geography
hh_per_scen_work_geog = pd.merge(hh_per_scen, taz_geog, left_on='Workplace TAZ', right_on='TAZ')
hh_per_base_work_geog = pd.merge(hh_per_base, taz_geog, left_on='Workplace TAZ', right_on='TAZ')

## Total People

In [359]:
print scen_name + ": "+ str(person_scen['personID'].count()).split('.')[0]
print base_name + ": "+ str(person_base['personID'].count()).split('.')[0]

2006 Survey: 10510
2014 Survey: 14250


## Average Household Size

In [360]:
print scen_name + ": "+ str(hh_scen['Household Size'].mean())
print base_name + ": "+ str(hh_base['Household Size'].mean())

2006 Survey: 2.21683189201
2014 Survey: 1.76130467013


## Household Size Distribution

In [361]:
df = pd.DataFrame([hh_scen.groupby('Household Size').count()['Household ID'],
                   hh_base.groupby('Household Size').count()['Household ID']]).T
df.columns=([scen_name,base_name])
df

,2006 Survey,2014 Survey
Household Size,,
1,1544,4253
2,1746,2444
3,621,723
4,613,497
5,158,123
6,49,37
7,5,15
8,5,2


In [362]:
# Distribution
df_dist = pd.DataFrame([df[scen_name]/person_scen.count()['Household ID'],
                   df[base_name]/person_base.count()['Household ID']]).T
df_dist
display_charts(df_dist, kind='bar', title='Household Size Distribution')

---

## Age Distribution

In [363]:
# 2006 survey age is continuous variable
# 2014 is categorical
from collections import OrderedDict


# lookup for 2014 names (which are averaged based on min and max of bin)
age_cat = OrderedDict([(2, '   <5'),
           (8, ' 5-11'),
           (14, '12-15'),
           (17, '16-17'),
          (21, '18-24'),
          (30, '25-34'),
          (40, '35-44'),
          (50, '44-54'),
          (60, '55-64'),
          (70, '65-74'),
          (80, '75-84'),
          (90, '85+')])


person_base['Age'] = [age_cat[x] for x in person_base['Age'].as_matrix()]

In [364]:
# convert 2006 into categorical
age_bins = [0,4,11,15,17,24,34,44,54,64,74,84,1000]
person_scen['Age'] = pd.cut(person_scen['Age'],age_bins,labels=age_cat.values())

In [365]:
df = pd.DataFrame([person_scen.groupby('Age').count()['Household ID'],
                   person_base.groupby('Age').count()['Household ID']]).T
df.columns=([scen_name,base_name])
df

,2006 Survey,2014 Survey
Age,,
<5,569,667
5-11,860,691
12-15,492,382
16-17,259,192
18-24,390,1911
25-34,878,2759
35-44,1572,1931
44-54,2002,1765
55-64,1840,2004


In [366]:
# Distribution
df_new = pd.DataFrame([df[scen_name]/df[scen_name].sum(),
             df[base_name]/df[base_name].sum()]).T
df_new

display_charts(df_new, title='Age', kind='bar')

---

## Workers Types

In [367]:
df = pd.DataFrame([person_scen.groupby('Worker Type').count()['personID'],
                   person_base.groupby('Worker Type').count()['personID']]).T
df.columns=([scen_name,base_name])
df

,2006 Survey,2014 Survey
Worker Type,,
Full-time worker,4285,6805
Not a worker,5160,6192
Part-time worker,1065,1253


---

** Distribution of Worker Types**

In [368]:
# Distribution
df_dist = pd.DataFrame([df[scen_name]/person_scen.count()['personID'],
                   df[base_name]/person_base.count()['personID']]).T

In [369]:
display_charts(df_dist, kind='bar', title='Worker Type Distribution')

---

## Workers by County

In [370]:
hh_per_scen_work_geog

,Age,Auto Distance to Work,Auto Time to Work,Expansion Factor_p,Household ID,Person Number,Student Type,Transit Pass,Usual Commute Mode,Worker Type,...,Household Size,Household TAZ,Household Vehicles,Household Workers,TAZ,TAD,OldDistric,County,District,New DistrictName
0,37,17.28,42.14,324.86,2001,1,0,1,515,Full-time worker,...,1,1309,1,1,515,11,11,King,4,Seattle CBD
1,54,11.66,32.44,255.38,10241,1,0,0,515,Full-time worker,...,2,1528,2,2,515,11,11,King,4,Seattle CBD
2,57,1.33,14.69,118.64,10463,1,0,0,515,Full-time worker,...,2,631,2,2,515,11,11,King,4,Seattle CBD
3,57,11.64,32.60,387.43,14519,1,0,0,515,Full-time worker,...,3,1560,4,3,515,11,11,King,4,Seattle CBD
4,56,7.28,18.06,235.72,14750,1,0,1,515,Full-time worker,...,2,70,0,2,515,11,11,King,4,Seattle CBD
5,35,3.04,19.06,151.89,17814,1,0,0,515,Full-time worker,...,2,592,3,2,515,11,11,King,4,Seattle CBD
6,56,1.91,14.78,129.55,17834,1,0,0,515,Full-time worker,...,2,424,1,2,515,11,11,King,4,Seattle CBD
7,50,14.35,34.85,304.73,19878,1,0,0,515,Full-time worker,...,2,1449,3,2,515,11,11,King,4,Seattle CBD
8,30,18.33,261.89,313.34,22635,2,0,1,515,Full-time worker,...,4,3659,2,2,515,11,11,King,4,Seattle CBD
9,37,4.41,16.20,734.04,23816,2,0,0,515,Full-time worker,...,2,284,0,2,515,11,11,King,4,Seattle CBD


In [388]:
# Workers by Workplace County

scen_workers = hh_per_scen_work_geog[hh_per_scen_work_geog['Worker Type'] <> 'Not a worker']
base_workers = hh_per_base_work_geog[hh_per_base_work_geog['Worker Type'] <> 'Not a worker']

df = scen_workers.groupby('County').count()['personID']
df=df[df.index<>0]
df=pd.DataFrame(df)
df.columns=[scen_name]

df_base = base_workers.groupby('County').count()['personID']
df_base=df_base[df_base.index<>0]
df[base_name]=df_base
df

# Also add in the CTPP here
# 2006-2010 CTPP

ctpp_workers = {'King':53625,'Kitsap':6475,'Pierce':15705,'Snohomish':15810}

df['CTPP 06-10'] = pd.DataFrame(ctpp_workers.values(),index=ctpp_workers.keys())

In [389]:
df

,2006 Survey,2014 Survey,CTPP 06-10
County,,,
King,3056,4488,53625
Kitsap,393,254,6475
Pierce,475,720,15705
Snohomish,549,626,15810


In [390]:
# Distribution
df_dist = pd.DataFrame([df[scen_name]/df[scen_name].sum(),
                       df[base_name]/df[base_name].sum(),
                       df['CTPP 06-10']/df['CTPP 06-10'].sum()]).T

In [391]:
display_charts(df_dist, kind='bar', title='Worker Distribution by County')

---

## Distance to Work

In [375]:
# Filter out non-workers
scen_df = (person_scen[person_scen['Worker Type'] <> 'Not a worker'])
base_df = (person_base[person_base['Worker Type'] <> 'Not a worker'])

df = pd.DataFrame([scen_df.groupby('Age').mean()['Auto Distance to Work'],
              base_df.groupby('Age').mean()['Auto Distance to Work']]).T
df.columns=([scen_name,base_name])
df.fillna(0,inplace=True)
df

,2006 Survey,2014 Survey
<5,0.000000,0
5-11,0.000000,0
12-15,0.000000,0
16-17,3.865185,1
18-24,8.319750,1
25-34,10.625236,1
35-44,10.303467,1
44-54,10.104780,1
55-64,9.807923,1
65-74,8.225459,1


In [376]:
display_charts(df, title='Distance to Work', kind='bar')

---

## Transit Pass Ownership

In [378]:
df = pd.DataFrame([person_scen.groupby('Transit Pass').mean()['Auto Distance to Work'],
              person_base.groupby('Transit Pass').mean()['Auto Distance to Work']]).T
df.columns=([scen_name,base_name])

In [379]:
df.index = ['No Pass', 'Transit Pass']
df

,2006 Survey,2014 Survey
No Pass,4.142860,NaN
Transit Pass,7.484511,1


---

## Auto Ownership

In [380]:
new_hh_scen = hh_scen[hh_scen.index>=0]
df = pd.DataFrame([new_hh_scen.groupby('Household Vehicles').count()['Household ID'],
                   hh_base.groupby('Household Vehicles').count()['Household ID']]).T
df.columns=([scen_name,base_name])
df=df[df.index>=0]
df

,2006 Survey,2014 Survey
Household Vehicles,,
0,219,1412
1,1576,3331
2,1914,2063
3,712,613
4,207,200
5,76,58
6,24,9
7,6,9
8,5,4


In [381]:
# Distribution
df_new = pd.DataFrame([df[scen_name]/df[scen_name].sum(),
             df[base_name]/df[base_name].sum()]).T

In [382]:
display_charts(df_new, title='Vehicles per Household', kind='bar', xlim=(0,6))

### Auto Ownership by Income

In [383]:
# Create common income ranges
def map_income(df, in_field, out_field):
    
    # Define categories
    incmap = {}
    for i in range(0, 20000):
        incmap.update({i: ' <20k'})
    for i in range(20000, 40000):
        incmap.update({i: '20k-40k'})
    for i in range(40000, 60000):
        incmap.update({i: '40k-60k'})
    for i in range(60000, 75000):
        incmap.update({i: '60k-75k'})
    for i in range(75000, 100000):
        incmap.update({i: '75k-100k'})
    for i in range(100000, 150000):
        incmap.update({i: '100k-150k'})
    for i in range(150000, int(df[in_field].max())+1):
        incmap.update({i: '>150k'})

    df[out_field] = df[in_field].map(incmap)
    return df

In [384]:
hh_scen = map_income(hh_scen, in_field='Household Income', out_field='Income')
hh_base = map_income(hh_base, in_field='Household Income', out_field='Income')

In [385]:
hh_base.groupby('Household Income').count()

,Expansion Factor,Household ID,Household Size,Household TAZ,Household Vehicles,Household Workers,Income
Household Income,,,,,,,
-1,2524,2524,2524,2524,2524,2524,0
5000,225,225,225,225,225,225,225
12500,14,14,14,14,14,14,14
17500,513,513,513,513,513,513,513
30000,455,455,455,455,455,455,455
37500,42,42,42,42,42,42,42
40000,642,642,642,642,642,642,642
62500,1035,1035,1035,1035,1035,1035,1035
87500,862,862,862,862,862,862,862


In [386]:
df_scen = pd.pivot_table(data=hh_scen, index='Household Vehicles', columns=['Income'], 
                    aggfunc='count', fill_value=False, margins=True)['Household ID']
df_base = pd.pivot_table(data=hh_base, index='Household Vehicles', columns=['Income'], 
                    aggfunc='count', fill_value=False, margins=True)['Household ID']

# Sort the columns
df_scen = df_scen[[' <20k','20k-40k','40k-60k','60k-75k',
                   '75k-100k','100k-150k','>150k']]
df_base = df_base[[' <20k','20k-40k','40k-60k','60k-75k',
                   '75k-100k','100k-150k','>150k']]

In [387]:
df_scen

Income,<20k,20k-40k,40k-60k,60k-75k,75k-100k,100k-150k,>150k
Household Vehicles,,,,,,,
0,100,66,29,7,3,1,1
1,195,411,382,128,183,83,26
2,41,165,337,183,478,307,166
3,8,47,129,65,183,132,74
4,1,12,25,23,57,43,16
5,0,5,11,5,17,18,6
6,1,2,2,1,3,4,3
7,0,0,0,0,2,2,1
8,0,0,0,0,1,3,0


---